In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invasive_ssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=40, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 48s 43ms/step - loss: 10.7192 - accuracy: 0.4295 - val_loss: 1.3779 - val_accuracy: 0.5109
Epoch 2/100
985/985 [==============================] - 63s 64ms/step - loss: 1.8887 - accuracy: 0.5161 - val_loss: 1.2095 - val_accuracy: 0.5055
Epoch 3/100
985/985 [==============================] - 57s 58ms/step - loss: 1.6468 - accuracy: 0.5680 - val_loss: 1.0804 - val_accuracy: 0.6082
Epoch 4/100
985/985 [==============================] - 66s 67ms/step - loss: 1.5297 - accuracy: 0.6193 - val_loss: 0.9018 - val_accuracy: 0.6562
Epoch 5/100
985/985 [==============================] - 57s 58ms/step - loss: 1.5980 - accuracy: 0.6341 - val_loss: 2.0388 - val_accuracy: 0.6290
Epoch 6/100
985/985 [==============================] - 56s 57ms/step - loss: 1.0381 - accuracy: 0.6601 - val_loss: 0.8070 - val_accuracy: 0.6979
Epoch 7/100
985/985 [==============================] - 58s 59ms/step - loss: 1.1082 - accuracy: 0.6613 - val_loss: 13.8118 - val_

Epoch 56/100
985/985 [==============================] - 57s 58ms/step - loss: 0.8577 - accuracy: 0.7494 - val_loss: 14.5738 - val_accuracy: 0.7229
Epoch 57/100
985/985 [==============================] - 57s 57ms/step - loss: 0.8798 - accuracy: 0.7397 - val_loss: 664.8566 - val_accuracy: 0.6899
Epoch 58/100
985/985 [==============================] - 54s 54ms/step - loss: 0.8179 - accuracy: 0.7490 - val_loss: 6415.5513 - val_accuracy: 0.7543
Epoch 59/100
985/985 [==============================] - 63s 64ms/step - loss: 0.6796 - accuracy: 0.7640 - val_loss: 1710.9465 - val_accuracy: 0.7541
Epoch 60/100
985/985 [==============================] - 55s 56ms/step - loss: 0.7980 - accuracy: 0.7633 - val_loss: 1.8588 - val_accuracy: 0.7759
Epoch 61/100
985/985 [==============================] - 56s 57ms/step - loss: 0.7421 - accuracy: 0.7561 - val_loss: 17.0709 - val_accuracy: 0.7750
Epoch 62/100
985/985 [==============================] - 54s 55ms/step - loss: 0.6851 - accuracy: 0.7612 - val_loss

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.3911037901869985
Test Accuracy: 0.5926264472882389


In [5]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 55s 50ms/step - loss: 1.0012 - accuracy: 0.7235 - val_loss: 1196.4908 - val_accuracy: 0.4384
Epoch 2/100
985/985 [==============================] - 58s 58ms/step - loss: 0.8458 - accuracy: 0.7337 - val_loss: 0.9320 - val_accuracy: 0.6566
Epoch 3/100
985/985 [==============================] - 42s 43ms/step - loss: 0.5347 - accuracy: 0.8039 - val_loss: 0.6323 - val_accuracy: 0.7695
Epoch 4/100
985/985 [==============================] - 44s 45ms/step - loss: 0.5154 - accuracy: 0.8182 - val_loss: 1.3514 - val_accuracy: 0.6037
Epoch 5/100
985/985 [==============================] - 54s 55ms/step - loss: 0.5396 - accuracy: 0.8041 - val_loss: 0.5253 - val_accuracy: 0.8030
Epoch 6/100
985/985 [==============================] - 46s 46ms/step - loss: 0.4793 - accuracy: 0.8256 - val_loss: 2.4249 - val_accuracy: 0.6598
Epoch 7/100
985/985 [==============================] - 56s 56ms/step - loss: 0.5909 - accuracy: 0.7939 - val_loss: 0.6627 - val

KeyboardInterrupt: 

In [17]:
from sklearn.metrics import accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.7316676823075361


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)